In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

from surprise import Dataset, Reader, SVD, SVDpp
from surprise.model_selection import cross_validate as surprise_cv, GridSearchCV
from auto_surprise.engine import Engine

In [ ]:

def load_data(path_train_tbr, path_train_ratings, path_submission=None):
    tbr = pd.read_csv(path_train_tbr)
    ratings = pd.read_csv(path_train_ratings)
    if 'sid_pid' in ratings.columns:
        ratings[['sid','pid']] = (
            ratings['sid_pid'].str.split('_', expand=True)
            .astype(int)
        )
        ratings = ratings[['sid', 'pid', 'rating']]
    submission = pd.read_csv(path_submission) if path_submission else None
    return tbr, ratings, submission

def integrate_wishlist(ratings, tbr):
    item_means = ratings.groupby('pid')['rating'].mean()
    global_mean = ratings['rating'].mean()
    tbr_unique = tbr.loc[
        ~tbr.set_index(['sid','pid']).index.isin(
            ratings.set_index(['sid','pid']).index
        )
    ].copy()
    tbr_unique['rating'] = tbr_unique['pid'].map(item_means).fillna(global_mean)
    return pd.concat([ratings, tbr_unique[['sid','pid','rating']]], ignore_index=True)

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def svdpp_cv(ratings, n_splits=5, n_factors=50, lr=0.005, reg=0.02, n_epochs=20):
    reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))
    data = Dataset.load_from_df(ratings[['sid','pid','rating']], reader)
    algo = SVDpp(n_factors=n_factors, lr_all=lr, reg_all=reg, n_epochs=n_epochs)
    res = surprise_cv(algo, data, measures=['RMSE'], cv=n_splits, verbose=False)
    return np.mean(res['test_rmse']), np.std(res['test_rmse'])

def tune_svdpp_params(ratings):
    reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))
    data = Dataset.load_from_df(ratings[['sid','pid','rating']], reader)
    engine = Engine(verbose=True, algorithms=['svd', 'svdpp'])
    best_algo, best_params, best_score, tasks = engine.train(
        data=data, 
        target_metric='test_rmse', 
        cpu_time_limit=60 * 60 * 11, 
        max_evals=100
    )
    return best_algo, best_params, best_score
    


In [39]:
tbr, ratings, _ = load_data('data/train_tbr.csv', 'data/train_ratings.csv')
ext = integrate_wishlist(ratings, tbr)

In [ ]:
sample_ext = ext.sample(n=300000, random_state=42)

In [43]:
print("Tuning SVD++ hyperparameters...")
best_algo, best_params, best_score = tune_svdpp_params(sample_ext)

Tuning SVD++ hyperparameters...


auto_surprise 0.1.9

Available CPUs: 20

Evaluating RMSE, MAE, MSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2262  1.2297  1.2256  1.2247  1.2210  1.2255  0.0028  
MAE (testset)     0.9706  0.9756  0.9712  0.9697  0.9679  0.9710  0.0026  
MSE (testset)     1.5036  1.5123  1.5021  1.4998  1.4909  1.5017  0.0068  
Fit time          0.23    0.30    0.30    0.29    0.30    0.28    0.03    
Test time         0.11    0.11    0.21    0.21    0.21    0.17    0.05    


Baseline loss: 1.2254557798655186

...

 87%|████████████████████████████████████▌     | 87/100 [10:59:59<1:38:37, 455.17s/trial, best loss: 0.8116688533798818]


----Done!----

Best algorithm: svd

Best hyperparameters: {'lr_bi': 0.008303336730098384, 'lr_bu': 0.0017951032960930578, 'lr_pu': 
0.0012571428840501644, 'lr_qi': 0.0028258871361842596, 'n_epochs': 49, 'n_factors': 11, 'reg_bi': 
0.025494651050976, 'reg_bu': 0.01914433779885645, 'reg_pu': 0.00027057462468805886, 'reg_qi': 0.07917123396754373}

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ Algorithm ┃ Hyperparameters                                                                ┃               Loss ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ svd       │ {'lr_bi': 0.008303336730098384, 'lr_bu': 0.0017951032960930578, 'lr_pu':       │ 0.8109545751173609 │
│           │ 0.0012571428840501644, 'lr_qi': 0.0028258871361842596, 'n_epochs': 49,         │                    │
│           │ 'n_factors': 11, 'reg_bi': 0.025494651050976, 'reg_bu': 0.01914433779885645,   │                    │
│           │ 'reg_pu': 0.00027057462468805886, 'reg_qi': 0.07917123396754373}               │                    │
│ svdpp     │ {'lr_bi': 0.0021172957971913086, 'lr_bu': 0.0005361269219239136, 'lr_pu':      │ 0.8116688533798818 │
│           │ 0.00013220170902706265, 'lr_qi': 0.01908712188715986, 'lr_yj':                 │                    │
│           │ 0.0006909032798958656, 'n_epochs': 195, 'n_factors': 96, 'reg_bi':             │                    │
│           │ 0.0010709861583502107, 'reg_bu': 0.0016926076579795353, 'reg_pu':              │                    │
│           │ 0.012374529985006917, 'reg_qi': 0.08191719323469421, 'reg_yj':                 │                    │
│           │ 0.009253842154775094}                                                          │                    │
└───────────┴────────────────────────────────────────────────────────────────────────────────┴────────────────────┘

In [ ]:
print(best_algo, best_params, best_score)

In [ ]:
print("Running with best SVD++ settings:")
mean_rmse, std_rmse = svdpp_cv(ext, best_params)

In [ ]:
print(f"SVD++ (optimized): RMSE = {mean_rmse:.4f} +/- {std_rmse:.4f}")